In [1]:

outputdebug = False 
def debug(msg):
    if outputdebug:
        print(msg)

class Node():
    def __init__(self, key):
        self.key = key
        self.left = None 
        self.right = None 




class AVLTree():
    def __init__(self, *args):
        self.node = None 
        self.height = -1  
        self.balance = 0; 
        
        if len(args) == 1: 
            for i in args[0]: 
                self.insert(i)
                
    def height(self):
        if self.node: 
            return self.node.height 
        else: 
            return 0 
    
    def is_leaf(self):
        return (self.height == 0) 
    
    def insert(self, key):
        tree = self.node
        
        newnode = Node(key)
        
        if tree == None:
            self.node = newnode 
            self.node.left = AVLTree() 
            self.node.right = AVLTree()
            debug("Inserted key [" + str(key) + "]")
        
        elif key < tree.key: 
            self.node.left.insert(key)
            
        elif key > tree.key: 
            self.node.right.insert(key)
        
        else: 
            debug("Key [" + str(key) + "] already in tree.")
            
        self.rebalance() 
        
    def rebalance(self):
        ''' 
        Rebalance a particular (sub)tree
        ''' 
        # key inserted. Let's check if we're balanced
        self.update_heights(False)
        self.update_balances(False)
        while self.balance < -1 or self.balance > 1: 
            if self.balance > 1:
                if self.node.left.balance < 0:  
                    self.node.left.lrotate() # we're in case II
                    self.update_heights()
                    self.update_balances()
                self.rrotate()
                self.update_heights()
                self.update_balances()
                
            if self.balance < -1:
                if self.node.right.balance > 0:  
                    self.node.right.rrotate() # we're in case III
                    self.update_heights()
                    self.update_balances()
                self.lrotate()
                self.update_heights()
                self.update_balances()


            
    def rrotate(self):
        # Rotate left pivoting on self
        debug ('Rotating ' + str(self.node.key) + ' right') 
        A = self.node 
        B = self.node.left.node 
        T = B.right.node 
        
        self.node = B 
        B.right.node = A 
        A.left.node = T 

    
    def lrotate(self):
        # Rotate left pivoting on self
        debug ('Rotating ' + str(self.node.key) + ' left') 
        A = self.node 
        B = self.node.right.node 
        T = B.left.node 
        
        self.node = B 
        B.left.node = A 
        A.right.node = T 
        
            
    def update_heights(self, recurse=True):
        if not self.node == None: 
            if recurse: 
                if self.node.left != None: 
                    self.node.left.update_heights()
                if self.node.right != None:
                    self.node.right.update_heights()
            
            self.height = max(self.node.left.height,
                              self.node.right.height) + 1 
        else: 
            self.height = -1 
            
    def update_balances(self, recurse=True):
        if not self.node == None: 
            if recurse: 
                if self.node.left != None: 
                    self.node.left.update_balances()
                if self.node.right != None:
                    self.node.right.update_balances()

            self.balance = self.node.left.height - self.node.right.height 
        else: 
            self.balance = 0 

    def delete(self, key):
        # debug("Trying to delete at node: " + str(self.node.key))
        if self.node != None: 
            if self.node.key == key: 
                debug("Deleting ... " + str(key))  
                if self.node.left.node == None and self.node.right.node == None:
                    self.node = None # leaves can be killed at will 
                # if only one subtree, take that 
                elif self.node.left.node == None: 
                    self.node = self.node.right.node
                elif self.node.right.node == None: 
                    self.node = self.node.left.node
                
                # worst-case: both children present. Find logical successor
                else:  
                    replacement = self.logical_successor(self.node)
                    if replacement != None: # sanity check 
                        debug("Found replacement for " + str(key) + " -> " + str(replacement.key))  
                        self.node.key = replacement.key 
                        
                        # replaced. Now delete the key from right child 
                        self.node.right.delete(replacement.key)
                    
                self.rebalance()
                return  
            elif key < self.node.key: 
                self.node.left.delete(key)  
            elif key > self.node.key: 
                self.node.right.delete(key)
                        
            self.rebalance()
        else: 
            return 

    def logical_predecessor(self, node):
        ''' 
        Find the biggest valued node in LEFT child
        ''' 
        node = node.left.node 
        if node != None: 
            while node.right != None:
                if node.right.node == None: 
                    return node 
                else: 
                    node = node.right.node  
        return node 
    
    def logical_successor(self, node):
        ''' 
        Find the smallese valued node in RIGHT child
        ''' 
        node = node.right.node  
        if node != None: # just a sanity check  
            
            while node.left != None:
                debug("LS: traversing: " + str(node.key))
                if node.left.node == None: 
                    return node 
                else: 
                    node = node.left.node  
        return node 

    def check_balanced(self):
        if self == None or self.node == None: 
            return True
        
        # We always need to make sure we are balanced 
        self.update_heights()
        self.update_balances()
        return ((abs(self.balance) < 2) and self.node.left.check_balanced() and self.node.right.check_balanced())  
        
    def inorder_traverse(self):
        if self.node == None:
            return [] 
        
        inlist = [] 
        l = self.node.left.inorder_traverse()
        for i in l: 
            inlist.append(i) 

        inlist.append(self.node.key)

        l = self.node.right.inorder_traverse()
        for i in l: 
            inlist.append(i) 
    
        return inlist 

    def display(self, level=0, pref=''):
        '''
        Display the whole tree. Uses recursive def.
        TODO: create a better display using breadth-first search
        '''        
        self.update_heights()  # Must update heights before balances 
        self.update_balances()
        if(self.node != None): 
            print('-' * level * 2, pref, self.node.key, "[" + str(self.height) + ":" + str(self.balance) + "]", 'L' if self.is_leaf() else ' ')   
            if self.node.left != None: 
                self.node.left.display(level + 1, '<')
            if self.node.left != None:
                self.node.right.display(level + 1, '>')
        



In [2]:
a = AVLTree()
for i in [5,6,9,8,10,7,4,1,3,2]:
    a.insert(i)
a.height

3

In [9]:
import time
import numpy as np


NUM = 1000

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.031202077865600586
1 번째 트리 높이:  11
1 - 1 번째 탐색, 찾는 숫자 30452 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 71171 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 32397 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 70512 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 6639 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.031281232833862305
2 번째 트리 높이:  11
2 - 1 번째 탐색, 찾는 숫자 80587 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 73807 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 20046 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 84151 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 438 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.031201839447021484
3 번째 트리 높이:  11
3 - 1 번째 탐색, 찾는 숫자 49717 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 17420 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 58158 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 76806 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 49717 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.031271934509277344
4 번째 트리 높이:  11
4 - 1 번째 탐색, 찾는 숫자 44886 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 4142 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 8100 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 62528 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 61539 탐색+삭제 시간: 0.0

5 번째 트리 생

In [10]:
import time
import numpy as np

NUM = 10000

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.35877561569213867
1 번째 트리 높이:  15
1 - 1 번째 탐색, 찾는 숫자 5171 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 47011 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 18313 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 62879 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 23840 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.38744449615478516
2 번째 트리 높이:  15
2 - 1 번째 탐색, 찾는 숫자 53249 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 3872 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 13902 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 37810 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 25163 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.4223909378051758
3 번째 트리 높이:  15
3 - 1 번째 탐색, 찾는 숫자 29258 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 53986 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 67489 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 60013 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 12341 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.7216897010803223
4 번째 트리 높이:  15
4 - 1 번째 탐색, 찾는 숫자 84970 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 22208 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 65874 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 71692 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 10491 탐색+삭제 시간: 0.0

5 번째 트리 생성 시

In [12]:
import time
import numpy as np

NUM = 100000

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  4.689314842224121
1 번째 트리 높이:  18
1 - 1 번째 탐색, 찾는 숫자 2457 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 6865 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 85883 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 59043 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 42917 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  5.24441123008728
2 번째 트리 높이:  18
2 - 1 번째 탐색, 찾는 숫자 35714 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 14178 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 80364 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 12756 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 93002 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  5.219849109649658
3 번째 트리 높이:  18
3 - 1 번째 탐색, 찾는 숫자 8178 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 32481 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 60320 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 9741 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 17712 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  4.537209987640381
4 번째 트리 높이:  18
4 - 1 번째 탐색, 찾는 숫자 66777 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 7565 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 58922 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 47344 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 41990 탐색+삭제 시간: 0.0

5 번째 트리 생성 시간:  4.4987

In [8]:
import time
import numpy as np

NUM = 1000000

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  32.91643190383911
1 번째 트리 높이:  19
1 - 1 번째 탐색, 찾는 숫자 76251 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 18445 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 43891 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 58711 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 87553 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  29.66185712814331
2 번째 트리 높이:  19
2 - 1 번째 탐색, 찾는 숫자 73937 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 15349 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 78570 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 43223 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 55409 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  31.681108713150024
3 번째 트리 높이:  19
3 - 1 번째 탐색, 찾는 숫자 19615 탐색+삭제 시간: 0.0009548664093017578
3 - 2 번째 탐색, 찾는 숫자 5101 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 85654 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 87858 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 11330 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  31.71998405456543
4 번째 트리 높이:  19
4 - 1 번째 탐색, 찾는 숫자 91858 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 53997 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 88071 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 82672 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 11131 탐색+삭제 시간: 0.0